In [1]:
import pandas as pd
import re

In [2]:
def predicting_txt_to_dataframe(path):    
    with open(path, 'r') as f:
        txt = str(f.read())
        txt_list = txt.split('\n')
    row_list = list()
    tmp_list = list()
    for row in txt_list:
        if row == '--------------------':
            tmp_list[0] = tmp_list[0].replace('article_id:', '')
            tmp_list[0] = int(tmp_list[0])
            row_list.append(tmp_list)
            tmp_list = list()
            continue
        if len(row) > 0:
            tmp_list.append(row)
    df = pd.DataFrame(row_list, columns=['article_id','text'])
    return df

In [3]:
df = predicting_txt_to_dataframe('/home/Danny/AI-CUP-2020/datasets/stage5/test.txt')
df

,article_id,text
0,0,醫師：最近人有沒有什麼不舒服？民眾：沒有。醫師：沒有喔。我們本來說要月，年底才打對不對？民眾...
1,1,醫師：要去上課嗎？民眾：沒有。這學期已經開始不上課了。醫師：啊今天的，呃，應該是昨天的抽血。...
2,2,醫師：後來那個吃藥沒有問題對不對？民眾：沒有。醫師：诶？你是打針的？民眾：對。醫師：阿身上有...
3,3,民眾：我自己昨天開始，一天吃一次一粒。醫師：一次一粒，減半量還好？民眾：阿可是……減到今天早...
4,4,醫師：喔，啊嘉明最近還好嗎？民眾：ＯＫ。醫師：啊今年的流感打了嗎？民眾：還沒。醫師：那我們今...
...,...,...
154,154,醫師：這個月還好嗎？民眾：就……醫師：這個月有去碰嗎？民眾：沒有，不去碰的時候就會覺得慾望非...
155,155,醫師：啊這個月還好嗎？民眾：還好啊。醫師：好，所以你反正天天吃啦齁？民眾：對。醫師：好，我們...
156,156,醫師：好那所以總之就是好，所以總之反正，總之就是……這這……就是反正你剩下的藥你就是……就是...
157,157,醫師：好，所以第一個月吃藥的狀況怎麼樣？民眾：嗯，吃藥狀況喔？醫師：嗯。民眾：就是吃了，一開...


In [4]:
entity_type_list = pd.read_csv('entity_type.csv')['entity_type'].tolist()
entity_type_list

['med_exam',
 'location',
 'time',
 'profession',
 'name',
 'money',
 'organization',
 'family',
 'ID',
 'clinical_event',
 'education',
 'contact',
 'others']

In [5]:
# contact_df = pd.read_csv('ground_truth/contact.csv')
# contact_list = contact_df['entity_text'].tolist()
# contact_list

In [20]:
def get_entity(article_id, text, entity_text, entity_type):
    output_str = str()
    source = 'dictionary'
    for match in re.finditer(entity_text, text, re.IGNORECASE):
        output_str += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(article_id, match.start(), match.end(), match.group(), entity_type, source)
    return output_str

In [21]:
output_str = '{}\t{}\t{}\t{}\t{}\t{}\n'.format('article_id', 'start_position', 'end_position', 'entity_text', 'entity_type', 'source')

In [22]:
# ckip
entity_type_list = ['education', 'ID', 'location', 'organization', 'profession']
for entity_type in entity_type_list:
    dictionary_path = 'ckip/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [24]:
# ground_truth
entity_type_list = ['clinical_event', 'contact', 'education', 'family', 'location', 'name', 'organization', 'profession', 'others', 'profession']
for entity_type in entity_type_list:
    dictionary_path = 'ground_truth/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [25]:
print(output_str)

article_id	start_position	end_position	entity_text	entity_type	source
25	647	651	高雄大學	education	dictionary
25	652	655	法律系	education	dictionary
144	107	110	電機所	education	dictionary
144	122	125	電機所	education	dictionary
144	120	125	清大電機所	education	dictionary
149	39	42	化學系	education	dictionary
8	1153	1158	56387	ID	dictionary
131	1213	1218	73846	ID	dictionary
45	2631	2633	高醫	location	dictionary
98	480	482	高醫	location	dictionary
117	1223	1225	高醫	location	dictionary
37	502	506	新光三越	location	dictionary
30	540	544	長庚醫院	location	dictionary
45	2292	2296	長庚醫院	location	dictionary
45	2844	2848	長庚醫院	location	dictionary
81	909	913	長庚醫院	location	dictionary
81	932	936	長庚醫院	location	dictionary
81	949	953	長庚醫院	location	dictionary
129	53	57	善化國小	location	dictionary
11	4919	4923	台南國小	location	dictionary
14	1011	1015	台大醫院	location	dictionary
8	216	218	台大	location	dictionary
14	1011	1013	台大	location	dictionary
45	2795	2797	台大	location	dictionary
68	1353	1355	台大	location	dictionary
97	558	560	台大	location	dicti

In [ ]:
with open('dictionary.tsv', 'w', encoding='utf-8') as f:
    f.write(output_str)

In [ ]:
df = pd.read_csv('dictionary.tsv', sep='\t')
df

In [ ]:
df = df.drop_duplicates()
df